# Experiments related to the personal contribution
EA algorithm to enhance client selection in the federated setting

# Setup
First, import of necessary libraries

In [1]:
import sys
import numpy as np

from models.model import LeNet5 
sys.path.append('../data/cifar100/')
from cifar100_loader import CIFAR100DataLoader
from utils.federated_utils import plot_metrics,test, plot_client_selection,save_data,load_data
from EA_algorithm import EA_algorithm

BATCH_SIZE = 64 #constant
GLOBAL_ROUNDS = 2000

# Data loading

In [2]:
#10% of the dataset kept for validation
data_loader = CIFAR100DataLoader(batch_size=BATCH_SIZE, validation_split=0.1, download=True, num_workers=4, pin_memory=True)
trainloader, validloader, testloader = data_loader.train_loader, data_loader.val_loader, data_loader.test_loader

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Crossover probability tuning
Fixing Nc=10, J=4 and number of generations/rounds=500, we evaluated different values of crossover probabilities to pick the best one for the final configuration of the EA algorithm. In particular, we tried the following values: 0.2,0.3,0.5 and 0.7.

In [ ]:
probabilities = [0.05,0.1,0.15,0.2]

max_val_acc = 0
max_test_acc = 0
best_probability = 0

for probability in probabilities:
  print('Starting test with probability: '+ str(probability))
  global_model = LeNet5()
  global_model,val_accuracies,val_losses,train_accuracies,train_losses,client_selection_count=EA_algorithm(500,5,2,10,probability,trainloader,validloader,0.1,0.001)
  max_validation_accuracy = max(val_accuracies)
  print('Max validation accuracy: '+ str(max_validation_accuracy))
  if(max_validation_accuracy > max_val_acc):
    max_val_acc = max_validation_accuracy
  test_accuracy = test(global_model,testloader)
  print("Test accuracy: ",test_accuracy)
  if(test_accuracy > max_test_acc):
    max_test_acc = test_accuracy
    best_proability = probability

print('Best probability found out of the test: '+ str(best_probability)+ ', with a test accuracy of: '+ str(max_test_acc))

# Experiments on non-iid distributions
Taking into account the same tests we have performed on the baseline to make a comparison, we will experiment with different non-iid distributions, with Nc=1,5,10,50. Hyperparameter tuning conducted in the same way as the previous experiments.

In [ ]:
#Hyperparameters tuning function
def hyperparameters_tuning(num_classes, rounds):
    print(f"Hyperparameter tuning for num_classes={num_classes}")
    lr_values = [0.1,0.05,0.01,0.005,0.001,0.0005,0.0001]
    wd_values = [0.001,0.0001]
    best_val_accuracy = 0
    best_setting = None
    for lr in lr_values:
        for wd in wd_values:
            print(f"Learning rate: {lr}, Weight decay: {wd}")
            model = LeNet5()
            #using the best values of population_size, num_clients and crossover_probability found in the previous step
            _,val_accuracies,val_losses,train_accuracies,train_losses,_=EA_algorithm(rounds,8,4,num_classes,0.5,trainloader,validloader,lr,wd)
            plot_metrics(train_accuracies, train_losses,val_accuracies, val_losses, f"Personal_contribution_tuning{num_classes}_lr_{lr}_wd_{wd}.png")
            max_val_accuracy = max(val_accuracies)
            print(f"Validation accuracy: {max_val_accuracy} with lr: {lr} and wd: {wd}")
            avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
            if avg_val_accuracy > best_val_accuracy:
                best_val_accuracy = avg_val_accuracy
                best_setting = (lr, wd)
    print(f"Best setting: {best_setting} with validation accuracy: {best_val_accuracy}")
    return best_setting

In [ ]:
num_classes = [1,5,10,50]

# Function to perform the training and testing for a given configuration
def run_experiment(num_classes, plot_suffix):
    print(f"Running experiment: num_classes={num_classes}")
    global_model = LeNet5()
    #100 rounds for hyperparameter tuning
    best_lr, best_wd = hyperparameters_tuning(num_classes = num_classes, rounds=100)

    global_model,val_accuracies,val_losses,train_accuracies,train_losses,client_selection_count=EA_algorithm(GLOBAL_ROUNDS,8,4,num_classes,0.5,trainloader,validloader,best_lr,best_wd)
    # Testing and plotting
    test_accuracy = test(global_model, testloader)
    plot_metrics(train_accuracies, train_losses, val_accuracies, val_losses, f"Contribution_{plot_suffix}_LR_{best_lr}_WD_{best_wd}.png")
    print(f"Test accuracy for num_classes={num_classes}: {test_accuracy}")
    plot_client_selection(client_selection_count, f"Client_selection_{plot_suffix}_LR_{best_lr}_WD_{best_wd}.png")
    # Save data for future analysis
    save_data(global_model, val_accuracies, val_losses, train_accuracies, train_losses, client_selection_count, f"Contribution_{plot_suffix}_LR_{best_lr}_WD_{best_wd}.pth")

# Main experiment loop
for num_classes in num_classes:  
    plot_suffix = f"num_classes_{num_classes}"
    run_experiment(num_classes, plot_suffix)